In [235]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
import json
import os
from sklearn.metrics.cluster import v_measure_score

In [319]:
import numpy as np
from sklearn.metrics import silhouette_score

def hamming_distance(x, y):
    # Calculate the Hamming distance between two strings of equal length
    distance = 0
    for i in range(len(x)):
        if x[i] != y[i]:
            distance += 1
    return distance
    
def kmeans(data, k, max_iterations=10000):
    # Initialize centroids with unique random choices from the data
    indices = np.random.choice(len(data), k, replace=False)
    centroids = [data[i] for i in indices]

    for _ in range(max_iterations):
        # Assign each data point to the nearest centroid
        clusters = [[] for i in range(k)]
        for point in data:
            distances = [hamming_distance(point, centroid) for centroid in centroids]
            cluster_index = np.argmin(distances)
            clusters[cluster_index] += [point]

        new_centroids = []
        
        for cluster in clusters:
            if cluster:
                new_centroid = ""
                for i in range(len(cluster[0])):
                    average_bit = int(round(np.mean([int(ohe[i]) for ohe in cluster])))
                    new_centroid += str(average_bit)
                new_centroids += [new_centroid]
            else:
                new_centroids += [centroids[i]]

        # Check if centroids == new centroids, if new, break
        if np.array_equal(centroids, new_centroids):
            break

        centroids = new_centroids

    return centroids, clusters

In [326]:
with open(os.getcwd() + '/db/item_class.json', 'r') as file:
    item_list = json.load(file)

In [322]:
file_path = './db/league_2.json'
with open(file_path, 'r') as file:
    data = [line for line in file]
#data is a list of frequent itemsets

774


In [323]:
data = list(filter(lambda x: x!= '\n', data))
for i in range(len(data)):
    data[i]= data[i][:-1]
data = [json.loads(_) for _ in data]
data = [_['items'] for _ in data]
#clean data

387


In [329]:
#calculate y for each item_set

#open the dictionary of item classes
file_path = './db/item_class.json'
with open(file_path, 'r') as file:
    itemmap = json.load(file)

y = []
for i in x:
    dp = {}
    res = ""
    high = 0
    for j in i:
        if j == '0':
            continue
        dp[itemmap[j]] = dp.get(itemmap[j], 0) + 1
        if dp[itemmap[j]] > high:
            res = itemmap[j]
            high = dp[itemmap[j]]
    y += [res]

#now, we want to map to the 311 items
x_data = []
for itemset in x:
    res = ""
    for i in item_list:
        if i not in itemset:
            res += "0"
        else:
            res += "1"
    x_data.append(res)
xtoy = {}
for i in range(len(x_data)):
    #map x to y in a dictionary
    xtoy[x_data[i]] = y[i] 


# Direct clustering and test

In [330]:
centroids, clusters = kmeans(x_data, 10)

calculate_silhouette_score(centroids, clusters)

centroidstoy = {}
for i in range(len(centroids)):
    dp = {}
    high = 0
    res = ''
    for ohe in clusters[i]:
        clas = xtoy[ohe]
        dp[clas] = dp.get(clas,0) + 1
        if dp[clas] > high:
            high = dp[clas]
            res = clas
    centroidstoy[centroids[i]] = res
        

# print(centroidstoy)

correct = 0
total = 0
for i in range(len(centroids)):
    for ohe in clusters[i]:
        total += 1
        if xtoy[ohe] == centroidstoy[centroids[i]]:
            correct += 1
print(total)



real = []
predicted = []
for i in range(len(clusters)):
    for j in clusters[i]:
        predicted += [centroidstoy[centroids[i]]]
        real += [xtoy[j]]
        
print('accuracy: ', correct/total)
print('v_measure: ', v_measure_score(real, predicted))

387
accuracy:  0.5322997416020672
v_measure:  0.26949926974167066


# Evolving this

## Normalising the count

In [332]:
type_count = {}
for i in item_list:
    clas = item_list[i]
    type_count[clas] = type_count.get(clas, 0) + 1
print(type_count)

{'Fighter': 85, 'Marksman': 55, 'Slayer': 30, 'Controller': 54, 'Tank': 30, 'Specialist': 5, 'Mage': 50, 'Unknown': 2}


In [333]:
#calculate y for each item_set

file_path = './db/item_class.json'
x_data = x
with open(file_path, 'r') as file:
    itemmap = json.load(file)
y = []
for i in x:
    dp = {}
    res = ""
    high = 0
    for j in i:
        if j == '0':
            continue
        dp[itemmap[j]] = dp.get(itemmap[j], 0) + 1/type_count[itemmap[j]] #item weighted against its rarity
        if dp[itemmap[j]] > high:
            res = itemmap[j]
            high = dp[itemmap[j]]
    y += [res]

#now, we want to map to the 311 items
x_data = []
for itemset in x:
    res = ""
    for i in item_list:
        if i not in itemset:
            res += "0"
        else:
            res += "1"
    x_data.append(res)
xtoy = {}
for i in range(len(x_data)):
    xtoy[x_data[i]] = y[i] 

In [334]:
centroids, clusters = kmeans(x_data, 20)

calculate_silhouette_score(centroids, clusters)

centroidstoy = {}
for i in range(len(centroids)):
    dp = {}
    high = 0
    res = ''
    for ohe in clusters[i]:
        clas = xtoy[ohe]
        dp[clas] = dp.get(clas,0) + 1
        if dp[clas] > high:
            high = dp[clas]
            res = clas
    centroidstoy[centroids[i]] = res
        

# print(centroidstoy)

correct = 0
total = 0
for i in range(len(centroids)):
    for ohe in clusters[i]:
        total += 1
        if xtoy[ohe] == centroidstoy[centroids[i]]:
            correct += 1
# print(total)

print(correct/total)

real = []
predicted = []
for i in range(len(clusters)):
    for j in clusters[i]:
        predicted += [centroidstoy[centroids[i]]]
        real += [xtoy[j]]
        

print('accuracy: ', correct/total)
print('v_measure: ', v_measure_score(real, predicted))

0.6356589147286822
accuracy:  0.6356589147286822
v_measure:  0.41200009347113004


## Finding best k

In [336]:
def k_test(k):
    centroids, clusters = kmeans(x_data, k)
    
    calculate_silhouette_score(centroids, clusters)
    
    centroidstoy = {}
    for i in range(len(centroids)):
        dp = {}
        high = 0
        res = ''
        for ohe in clusters[i]:
            clas = xtoy[ohe]
            dp[clas] = dp.get(clas,0) + 1
            if dp[clas] > high:
                high = dp[clas]
                res = clas
        centroidstoy[centroids[i]] = res
    
    # centroidstoy
    
    correct = 0
    total = 0
    for i in range(len(centroids)):
        for ohe in clusters[i]:
            total += 1
            if xtoy[ohe] == centroidstoy[centroids[i]]:
                correct += 1
    
    # print("accuracy: ", correct/total)
    
    real = []
    predicted = []
    for i in range(len(clusters)):
        for j in clusters[i]:
            predicted += [centroidstoy[centroids[i]]]
            real += [xtoy[j]]
    # print(real)
    # print(predicted)
    
    # print("accuracy: ", correct/total)
    # print("v_measure: ", v_measure_score(real, predicted))
    return (correct/total, v_measure_score(real, predicted))

accuracy_control, v_measure_control = k_test(10)
print(accuracy_control, v_measure_control)
best = 10
score = v_measure_control
optimal = (accuracy_control, v_measure_control)
for i in range(2, 200):
    accuracy_new, v_measure_new = k_test(i)
    gain = (v_measure_new/score)
    if gain > 1:
        optimal = (accuracy_new, v_measure_new)
        best = i
        score = v_measure_new
print(f'optimal k is: {best} with accuracy {optimal[0]} and v_measure {optimal[1]}')


0.5322997416020672 0.2622813176677009
optimal k is: 177 with accuracy 0.9095607235142119 and v_measure 0.7945470213526517
